In [106]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.svm import SVC  
import scipy.stats as stats
from sklearn import metrics
from sklearn import ensemble
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import Lasso
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [4]:
train_file = pd.read_csv("C:/Users/mruna/Desktop/Kaggle/Dont_overfit/train.csv")
train_file.head()

,id,target,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,0,1.0,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,1,0.0,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,2,1.0,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,3,1.0,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,4,1.0,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


In [5]:
train_file.describe()

,id,target,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
count,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,...,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
mean,124.500000,0.640000,0.023292,-0.026872,0.167404,0.001904,0.001588,-0.007304,0.032052,0.078412,...,0.044652,0.126344,0.018436,-0.012092,-0.065720,-0.106112,0.046472,0.006452,0.009372,-0.128952
std,72.312977,0.480963,0.998354,1.009314,1.021709,1.011751,1.035411,0.955700,1.006657,0.939731,...,1.011416,0.972567,0.954229,0.960630,1.057414,1.038389,0.967661,0.998984,1.008099,0.971219
min,0.000000,0.000000,-2.319000,-2.931000,-2.477000,-2.359000,-2.566000,-2.845000,-2.976000,-3.444000,...,-2.804000,-2.443000,-2.757000,-2.466000,-3.287000,-3.072000,-2.634000,-2.776000,-3.211000,-3.500000
25%,62.250000,0.000000,-0.644750,-0.739750,-0.425250,-0.686500,-0.659000,-0.643750,-0.675000,-0.550750,...,-0.617000,-0.510500,-0.535750,-0.657000,-0.818500,-0.821000,-0.605500,-0.751250,-0.550000,-0.754250
50%,124.500000,1.000000,-0.015500,0.057000,0.184000,-0.016500,-0.023000,0.037500,0.060500,0.183500,...,0.067500,0.091000,0.057500,-0.021000,-0.009000,-0.079500,0.009500,0.005500,-0.009000,-0.132500
75%,186.750000,1.000000,0.677000,0.620750,0.805000,0.720000,0.735000,0.660500,0.783250,0.766250,...,0.797250,0.804250,0.631500,0.650250,0.739500,0.493000,0.683000,0.794250,0.654250,0.503250
max,249.000000,1.000000,2.567000,2.419000,3.392000,2.771000,2.901000,2.793000,2.546000,2.846000,...,2.865000,2.801000,2.736000,2.596000,2.226000,3.131000,3.236000,2.626000,3.530000,2.771000


In [6]:
train_file_corr = train_file.drop(['id', 'target'], axis = 1).corr()
print(train_file_corr)

            0         1         2         3         4         5         6  \
0    1.000000 -0.003871 -0.010487 -0.047819  0.013967  0.070091 -0.022537   
1   -0.003871  1.000000  0.013573 -0.018884  0.086743 -0.028023 -0.032914   
2   -0.010487  0.013573  1.000000 -0.031620  0.088980 -0.050191  0.024674   
3   -0.047819 -0.018884 -0.031620  1.000000  0.025255  0.172058  0.157954   
4    0.013967  0.086743  0.088980  0.025255  1.000000 -0.013072 -0.030131   
5    0.070091 -0.028023 -0.050191  0.172058 -0.013072  1.000000 -0.002426   
6   -0.022537 -0.032914  0.024674  0.157954 -0.030131 -0.002426  1.000000   
7    0.002832 -0.066416  0.035260 -0.018158  0.037315  0.087932 -0.035602   
8   -0.060031 -0.027201 -0.000094  0.058736 -0.186559  0.013425 -0.043699   
9   -0.052356  0.035512  0.009949 -0.087360  0.031577  0.055556 -0.014648   
10   0.059263  0.020358  0.015158  0.095171 -0.008063  0.022024  0.139278   
11  -0.014094  0.003761 -0.029568  0.007442  0.007968 -0.010794 -0.199293   

In [7]:
train_file_corr[train_file_corr['1'] >= 0.5 ]

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1,-0.003871,1.0,0.013573,-0.018884,0.086743,-0.028023,-0.032914,-0.066416,-0.027201,0.035512,...,-0.0063,-0.077365,-0.021583,-0.054171,-0.046174,0.04282,-0.127499,0.065883,0.05547,-0.056612


In [8]:
y = train_file['target']
drops = ['id', 'target']
X = train_file
X.drop(drops, inplace=True, axis=1)
X.head()

#vif = pd.DataFrame()
#vif["features"] = train_file.columns
#vif["vif_score"] = [variance_inflation_factor(train_file.values, i) for i in range(train_file.shape[1])]

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,-2.246,1.825,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,0.004,-0.291,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,0.137,0.183,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,0.503,0.274,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,-0.012,2.198,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


In [9]:
#y

In [10]:
# recursive feature elimination

logreg = LogisticRegression()
# create the RFE model for the svm classifier 
# and select attributes
rfe = RFE(logreg, 275)
rfe1 = rfe.fit(X, y)
# print summaries for the selection of attributes
print(rfe1.support_)
print(rfe1.ranking_)

[ True  True False  True  True  True False  True  True  True False  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True False  True  True  True  True  True  True  True  True
  True  True  True False  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True  True  True
  True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True  True  True  T

In [11]:
features_bool = np.array(rfe1.support_)
features = np.array(X.columns)
result = features[features_bool]
print(result)
X = X[result]

['0' '1' '3' '4' '5' '7' '8' '9' '11' '13' '14' '15' '16' '17' '18' '19'
 '20' '21' '22' '23' '24' '25' '26' '28' '29' '30' '31' '32' '33' '34'
 '35' '36' '37' '38' '40' '41' '42' '43' '44' '45' '46' '47' '49' '50'
 '51' '52' '53' '54' '55' '56' '57' '58' '59' '60' '62' '63' '64' '65'
 '66' '67' '68' '69' '70' '71' '72' '73' '74' '75' '76' '77' '78' '79'
 '80' '81' '82' '83' '84' '85' '86' '87' '88' '89' '90' '91' '92' '93'
 '94' '95' '96' '97' '98' '99' '100' '101' '102' '103' '105' '106' '107'
 '108' '110' '111' '112' '113' '114' '115' '116' '117' '118' '119' '120'
 '121' '122' '123' '125' '127' '128' '129' '130' '131' '132' '133' '134'
 '135' '136' '137' '138' '139' '141' '142' '143' '145' '146' '147' '148'
 '149' '150' '151' '152' '153' '154' '155' '156' '157' '159' '160' '161'
 '162' '163' '164' '165' '166' '167' '168' '169' '170' '171' '172' '174'
 '175' '176' '177' '178' '179' '180' '181' '182' '183' '184' '185' '187'
 '188' '189' '190' '191' '192' '193' '194' '195' '196' '197' 

In [12]:
X.shape

(250, 275)

In [13]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### Logistic regression model

In [14]:
# instantiate the model (using the default parameters)
log_clf = LogisticRegression()
# fit the model with data
log_clf.fit(X_train,y_train)
y_pred=log_clf.predict(X_test)

In [15]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[17, 11],
       [13, 34]], dtype=int64)

In [16]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.68
Precision: 0.7555555555555555
Recall: 0.723404255319149


In [17]:
y_pred

array([1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1.,
       1., 0., 1., 1., 0., 1., 0.])

### Random forest model

In [22]:
forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [23]:
y_pred = forest_clf.predict(X_test)

In [24]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[ 9, 19],
       [11, 36]], dtype=int64)

In [25]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.6
Precision: 0.6545454545454545
Recall: 0.7659574468085106


### SVM model

In [57]:
svc_clf = SVC(kernel='linear', C = 100)  
svc_clf.fit(X_train, y_train)  

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [58]:
y_pred = svc_clf.predict(X_test)

In [59]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[15, 13],
       [ 5, 42]], dtype=int64)

In [60]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.76
Precision: 0.7636363636363637
Recall: 0.8936170212765957


### Lasso model

In [79]:
lasso_clf = Lasso(alpha=0.03, tol=0.01, selection='random', random_state=42)
#lasso_clf.fit(X_train, y_train)

Lasso(alpha=0.03, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=42,
   selection='random', tol=0.01, warm_start=False)

In [109]:
# splits = 10
# folds = RepeatedStratifiedKFold(n_splits=splits, n_repeats=20, random_state=42)
# oof_preds = np.zeros(X_train.shape[0])
# sub_preds = np.zeros(X_test.shape[0])

# for fold_, (trn_, val_) in enumerate(folds.split(X_train, y_train)):
#     trn_x, trn_y = X_train.iloc[trn_], y_train[trn_]
#     val_x, val_y = X_train.iloc[val_], y_train[val_]

# model = RFECV(lasso_clf, step=1, cv=(splits - 1))
# model.fit(X_train, y_train)
# oof_preds[val_] = model.predict(val_x).clip(0, 1)
# sub_preds += model.predict(X_test).clip(0, 1) / splits / 20 #folds.n_splits

In [115]:
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# print("Precision:",metrics.precision_score(y_test, y_pred))
# print("Recall:",metrics.recall_score(y_test, y_pred))

### GRID search for random forest

In [26]:
grid_1 = { 'bootstrap': [True],
           'max_depth': [80, 90, 100, 110],
        'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
   'min_samples_split': [8, 10, 12],
        'n_estimators': [100, 200, 300, 1000]
           }
rf = RandomForestClassifier()
grid_search = GridSearchCV(rf, param_grid = grid_1, n_jobs=-1, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110], 'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5], 'min_samples_split': [8, 10, 12], 'n_estimators': [100, 200, 300, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
grid_search.best_score_

0.6514285714285715

In [28]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}

In [31]:
rfclf1 = RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 8,
                               n_estimators= 100)
rfclf1.fit(X_train, y_train)
y_pred = rfclf1.predict(X_test)

In [32]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.64
Precision: 0.6351351351351351
Recall: 1.0


X_train.shape
X_test.shape

### GBM model

In [33]:
tree_clf = ensemble.GradientBoostingClassifier()
tree_clf.fit(X_train, y_train)
y_pred = tree_clf.predict(X_test)

In [34]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[ 9, 19],
       [ 3, 44]], dtype=int64)

In [35]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.7066666666666667
Precision: 0.6984126984126984
Recall: 0.9361702127659575


In [69]:
estimators = []
#estimators.append(('logistic', log_clf))
#estimators.append(('gbm', tree_clf))
estimators.append(('randomforest', rfclf1))
estimators.append(('svm', svc_clf))
# create the ensemble model
ensemble = VotingClassifier(estimators)

In [70]:
results = cross_val_score(ensemble, X, y, cv=8)
print(results.mean())
results

C:\Users\mruna\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mruna\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mruna\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mruna\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

0.7521421370967742


C:\Users\mruna\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([0.78125   , 0.6875    , 0.74193548, 0.90322581, 0.64516129,
       0.70967742, 0.77419355, 0.77419355])

### Cross validation

In [173]:
# Perform 6-fold cross validation
#logreg
scores = cross_val_score(tree_clf, X, y, cv=10)
print("Cross-validated scores:", scores)

Cross-validated scores: [0.76 0.6  0.68 0.68 0.6  0.8  0.76 0.68 0.64 0.68]
